<a href="https://colab.research.google.com/github/yzy-bupt/code/blob/main/humanparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 128215 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.2-1build1_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-1build1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.2-1build1_amd64.deb ...
Unpacking libgflags-dev (2.2.2-1build1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.4.0-1build1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.4.0-1build1) ...
Selecting previously unselected package libleveldb1d:amd64.
Preparing to unpack .../04-libleveldb1d_1.22-3ubuntu2_amd64.deb ...
Unpacking libleveldb1d:amd64 (1.22-3ubuntu2) ...
Selecting previously unselected package libleveldb-dev

In [4]:
!. ./openpose/models/getModels.sh

--2023-03-06 05:41:05--  http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/body_25/pose_iter_584000.caffemodel
Resolving posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)... 128.2.220.57
Connecting to posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)|128.2.220.57|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104715850 (100M) [text/plain]
Saving to: ‘pose/body_25/pose_iter_584000.caffemodel’

pose_iter_584000.ca 100%[===================>]  99.86M  9.31MB/s    in 16s     

2023-03-06 05:41:22 (6.29 MB/s) - ‘pose/body_25/pose_iter_584000.caffemodel’ saved [104715850/104715850]

--2023-03-06 05:41:22--  http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/coco/pose_iter_440000.caffemodel
Resolving posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)... 128.2.220.57
Connecting to posefs1.perception.cs.cmu.edu (posefs1.perception.cs.cmu.edu)|128.2.220.57|:80... connected.
HTTP request sent, awaiting response... 200 OK
L

In [5]:
# for storing input image
!mkdir ./image_path
# copy official provided data to image_path, you may need to download and unzip it in advance
# !cp ./test/image/000* ./image_path/
# create directories for generated results of OpenPose
!mkdir ./json_path
!mkdir ./img_path

In [1]:
# go to openpose directory
# %cd openpose
# run openpose.bin
!./build/examples/openpose/openpose.bin  --video ../img_path/4.avi --hand --disable_blending --display 0 --write_json ../json_path --write_video ../img_path/v.avi --num_gpu 0 --num_gpu_start 0

/bin/bash: ./build/examples/openpose/openpose.bin: No such file or directory


In [32]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# run openpose.bin
!./build/examples/openpose/openpose.bin --image_dir ../image_path --hand --disable_blending --display 0 --write_json ../json_path --write_images ../img_path --num_gpu 1 --num_gpu_start 0

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
F0306 07:23:07.078251 41379 cudnn_conv_layer.cpp:53] Check failed: status == CUDNN_STATUS_SUCCESS (1 vs. 0)  CUDNN_STATUS_NOT_INITIALIZED
*** Check failure stack trace: ***
    @     0x7f43c0d5d1c3  google::LogMessage::Fail()
    @     0x7f43c0d6225b  google::LogMessage::SendToLog()
    @     0x7f43c0d5cebf  google::LogMessage::Flush()
    @     0x7f43c0d5d6ef  google::LogMessageFatal::~LogMessageFatal()
    @     0x7f43c0983115  caffe::CuDNNConvolutionLayer<>::LayerSetUp()
    @     0x7f43c0a67e8d  caffe::Net<>::Init()
    @     0x7f43c0a6a3a5  caffe::Net<>::Net()
    @     0x7f43c1412c6c  op::NetCaffe::initializationOnThread()
    @     0x7f43c13f7c23  op::HandExtractorCaffe::netInitializationOnThread()
    @     0x7f43c13f9255  op::HandExtractorNet::initializationOnThread()
    @     0x7f43c1458887  op::Worker<>::initializationOnThreadNoException()
    @     0x7f43c14589d8  op::SubThread<>::initializationOnThrea

In [26]:
!nvidia-smi

Mon Mar  6 07:20:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

22222222222222222222222

In [1]:
%%bash
FILE_NAME='./CIHP_pgn.zip'
FILE_ID='1Mqpse5Gen4V4403wFEpv3w3JAsWw2uhk'

curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=$FILE_ID" > /dev/null
CODE="$(awk '/_warning_/ {print $NF}' /tmp/cookie)"  
curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=$FILE_ID" -o $FILE_NAME

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1172M  100 1172M    0     0   149M      0  0:00:07  0:00:07 --:--:--  143M


In [2]:
!unzip CIHP_pgn.zip

Archive:  CIHP_pgn.zip
   creating: CIHP_pgn/
  inflating: CIHP_pgn/checkpoint     
  inflating: CIHP_pgn/model.ckpt-593292.data-00000-of-00001  
  inflating: CIHP_pgn/model.ckpt-593292.index  
  inflating: CIHP_pgn/model.ckpt-593292.meta  


In [3]:
!git clone https://github.com/Engineering-Course/CIHP_PGN.git

Cloning into 'CIHP_PGN'...
remote: Enumerating objects: 4402, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 4402 (delta 14), reused 21 (delta 6), pack-reused 4364
Receiving objects: 100% (4402/4402), 852.01 KiB | 14.69 MiB/s, done.
Resolving deltas: 100% (720/720), done.


In [4]:
%cd CIHP_PGN/

/content/CIHP_PGN


In [5]:
!mkdir -p ./checkpoint
!mkdir -p ./datasets/images
# You also need to download dataset provided or use your own images
!mv ../CIHP_pgn ./checkpoint/CIHP_pgn
# !cp ./datasets/CIHP/images/00* ./datasets/images

！！！！！！！！！！！！！！

In [36]:
!sudo update-alternatives --config python3

There is only one alternative in link group python3 (providing /usr/bin/python3): /usr/bin/python3.8
Nothing to configure.


In [38]:
!sudo apt-get install python3-pip
!python -m pip install --upgrade pip
!pip install matplotlib opencv-python==4.2.0.32 Pillow scipy tensorflow==1.15
!pip install ipykernel

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (20.0.2-5ubuntu1.8).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached opencv_python-4.2.0.32-cp38-cp38-manylinux1_x86_64.whl (28.2 MB)
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10

In [39]:
!apt-get install python3.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython3.6-minimal libpython3.6-stdlib python3.6-minimal
Suggested packages:
  python3.6-venv binfmt-support
The following NEW packages will be installed:
  libpython3.6-minimal libpython3.6-stdlib python3.6 python3.6-minimal
0 upgraded, 4 newly installed, 0 to remove and 22 not upgraded.
Need to get 4,294 kB of archives.
After this operation, 22.1 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.6-minimal amd64 3.6.15-1+focal3 [569 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.6-minimal amd64 3.6.15-1+focal3 [1,718 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.6-stdlib amd64 3.6.15-1+focal3 [1,758 kB]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.6 amd64 3.

In [70]:
!sudo apt-get install python3-pip
!python -m pip install --upgrade pip
!pip install matplotlib opencv-python==4.2.0.32 Pillow scipy tensorflow==1.15
!pip install ipykernel

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (20.0.2-5ubuntu1.8).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 12.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.2.2
    Uninstalling pip-21.2.2:
      Successfully uninstalled pip-21.2.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 13.1 MB/s            
     |████████████████████████████████| 28.2 MB 65.3 MB/s            
     |████████████████████████████████| 3.1 MB 64.8 MB/s            
     |████████████████████████████████| 25.9 MB 1.4 MB/s             
     |████████████████████████████████| 412.3 MB 19 kB/s              
     |██████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 121 kB 15.3 MB/s            
     |████████████████████████████████| 783 kB 84.5 MB/s            
     |████████████████████████████████| 75 kB 4.6 MB/s             
     |████████████████████████████████| 427 kB 68.0 MB/s            
     |████████████████████████████████| 130 kB 65.7 MB/s            
     |████████████████████████████████| 1.4 MB 61.5 MB/s            
     |████████████████████████████████| 1.1 MB 66.2 MB/s            
     |████████████████████████████████| 59 kB 5.5 MB/s             
     |████████████████████████████████| 386 kB 79.0 MB/s            
     |████████████████████████████████| 1.0 MB 51.1 MB/s            
     |████████████████████████████████| 86 kB 6.2 MB/s             
     |████████████████████████████████| 109 kB 64.3 MB/s            


In [51]:
!sudo apt-get install python3-pip
#!python3 -m pip install
!python3.6 -m pip install matplotlib opencv-python==4.2.0.32 Pillow scipy tensorflow==1.15
!python3.6 -m pip install ipykernel

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (20.0.2-5ubuntu1.8).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/pip/__main__.py", line 29, in <module>
    from pip._internal.cli.main import main as _main
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/main_parser.py", line 9, 

In [58]:
# !update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6

update-alternatives: --install needs <link> <name> <path> <priority>

Use 'update-alternatives --help' for program usage information.


！！！！！！！！！！！！！！！！！！！

In [8]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2023-03-08 02:07:36--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2023-03-08 02:07:36--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 33.5M 2s
    50K .......... .......... .......... .......... ..........  0%

In [ ]:
!python --version

In [9]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    sqlite-3.38.5              |       hc218d9a_0         1.5 MB
    idna-3.3                   |     pyhd3eb1b0_0          55 KB
    wheel-0.37.1               |     pyhd3eb1b0_0          31 KB
    ca-certificates-2023.01.10 |       h06a4308_0         127 KB
    ncurses-6.3                |       h7f8727e_2         1.0 MB
    python-3.6.13              |       h12debd9_1        32.5 MB
    pycparser-2.21             |     pyhd3eb1b0_0          94 KB
    tqdm-4.63.0                |     pyhd3eb1b0_0          80 KB
    colorama-0.4.4             |     pyhd3eb1b0_0          21 KB
    setuptools-58.0.4          |   py36h06a4308_0         979 KB
    urllib3-1.26.8             |     pyhd3eb1b0_0   

sqlite-3.38.5        |  1.5 MB | ########## | 100% 
idna-3.3             |   55 KB | ########## | 100% 
wheel-0.37.1         |   31 KB | ########## | 100% 
ca-certificates-2023 |  127 KB | ########## | 100% 
ncurses-6.3          |  1.0 MB | ########## | 100% 
python-3.6.13        | 32.5 MB | ########## | 100% 
pycparser-2.21       |   94 KB | ########## | 100% 
tqdm-4.63.0          |   80 KB | ########## | 100% 
colorama-0.4.4       |   21 KB | ########## | 100% 
setuptools-58.0.4    |  979 KB | ########## | 100% 
urllib3-1.26.8       |  100 KB | ########## | 100% 
pyopenssl-22.0.0     |   49 KB | ########## | 100% 
libffi-3.3           |   54 KB | ########## | 100% 
charset-normalizer-2 |   33 KB | ########## | 100% 
pycosat-0.6.3        |  107 KB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
six-1.16.0           |   19 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
ruamel_yaml-0.15.100 |  268 KB | ########## | 100% 
yaml-0.2.5  

In [10]:
!python --version # now returns Python 3.6.13 :: Anaconda, Inc.

Python 3.6.13 :: Anaconda, Inc.


In [11]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython']

In [12]:
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))
_ = (sys.path.append("/usr/local/lib/python3.6/dist-packages"))



In [13]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages']

In [16]:
!sudo update-alternatives --config python3

There is only one alternative in link group python3 (providing /usr/bin/python3): /usr/bin/python3.8
Nothing to configure.


In [15]:
!sudo apt-get install python3-pip
!python -m pip install --upgrade pip
!pip install matplotlib opencv-python==4.2.0.32 Pillow scipy tensorflow==1.15
!pip install ipykernel

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-pip python3-setuptools python3-wheel
0 upgraded, 4 newly installed, 0 to remove and 22 not upgraded.
Need to get 2,389 kB of archives.
After this operation, 4,933 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python-pip-whl all 20.0.2-5ubuntu1.8 [1,805 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-setuptools all 45.2.0-1ubuntu0.1 [330 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python3-wheel all 0.34.2-1ubuntu0.1 [23.9 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python3-pip all 20.0.2-5ubuntu1.8 [231 kB]
Fetched 2,389 kB in 0s (14

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 121 kB 18.6 MB/s            
     |████████████████████████████████| 783 kB 56.0 MB/s            
     |████████████████████████████████| 130 kB 52.1 MB/s            
     |████████████████████████████████| 75 kB 5.3 MB/s             
     |████████████████████████████████| 427 kB 67.3 MB/s            
     |████████████████████████████████| 1.1 MB 63.5 MB/s            
     |████████████████████████████████| 59 kB 7.3 MB/s             
     |████████████████████████████████| 386 kB 74.1 MB/s            
     |████████████████████████████████| 1.4 MB 72.7 MB/s            
     |████████████████████████████████| 1.0 MB 70.6 MB/s            
     |████████████████████████████████| 86 kB 7.1 MB/s             
     |████████████████████████████████| 109 kB 63.1 MB/s            


In [ ]:
!python ./inf_pgn.py 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(inpu

In [3]:
!cd ..